In [7]:
#导包
from netCDF4 import Dataset
from tensorflow.keras import layers, models
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, LayerNormalization,Input, Conv3D, TimeDistributed, Flatten
from tensorflow.keras.layers import Concatenate,Conv2D,TimeDistributed, MaxPooling2D, Input, MaxPooling3D
from tensorflow.keras.layers import  Reshape,multiply
from tensorflow.keras.layers import Layer,Lambda,Dot,ReLU,LSTM, Dense, Dropout, Activation, Flatten,Attention
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.losses import MeanSquaredError
from sklearn.metrics import r2_score,accuracy_score,precision_score
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras import regularizers
import tensorflow as tf
from datetime import datetime
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import models, layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import tensorflow.keras as keras
from tensorflow.keras import layers, Sequential, regularizers
# calculate RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas as pd 
import tensorflow as tf
import gc 
import gzip
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import match
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os 
%matplotlib inline

In [8]:
len(tf.config.experimental.list_physical_devices('GPU'))

1

# 读取数据

## SSH

In [9]:
data_ssh = nc.Dataset(r'D:/data/SSH/SSH_2005-2015_month.nc')

In [10]:
time_ssh = data_ssh['time'][:]
print(len(time_ssh))
ssh_lat = data_ssh['lat'][:].data
ssh_lon = data_ssh['lon'][:].data
ssh = data_ssh['ssh'][:].data
print(ssh.shape)

132


In [13]:
np.nanmax(ssh),np.nanmin(ssh)

(2.20051, -0.43673334)

## SSS

In [14]:
data_sss = nc.Dataset(r'D:/data/SSS/SSS_2005-2020_month.nc')

In [16]:
time_sss = data_sss['time'][:132]
print(len(time_sss))
sss_lat = data_sss['lat'][:].data
sss_lon = data_sss['lon'][:].data
sss = data_sss['sos'][:132].data 
print(sss.shape)
sss = np.squeeze(sss)
print(sss.shape)

In [18]:
np.nanmax(sss),np.nanmin(sss)

(36.072178, 13.604756)

## SSW

In [19]:
data_ssw = nc.Dataset(r'D:/data/SSW/SSW_2005-2018_month.nc')

In [20]:
time_ssw = data_ssw['time'][:132]
print(len(time_ssw))
lat_ssw = data_ssw['lat'][:].data
lon_ssw = data_ssw['lon'][:].data

132


In [22]:
uwnd = data_ssw['uwnd'][:132,314:594,440:1040].data  
vwnd = data_ssw['vwnd'][:132,314:594,440:1040].data
print(uwnd.shape)
print(vwnd.shape)

In [24]:
np.nanmax(uwnd),np.nanmin(uwnd),np.nanmax(vwnd),np.nanmin(vwnd)

(12.433594, -11.7988205, 8.8359995, -13.645481)

## SST

In [25]:
data_sst = nc.Dataset('D:/data/SST/SST_2005-2015_month.nc','r')

In [26]:
time_sst = data_sst['time'][:]
print(len(time_sst))
sst_lat = data_sst['lat'][:].data
sst_lon = data_sst['lon'][:].data

132


In [27]:
sst = data_sst['sst'][:].data
print(ssh.shape)

(132, 280, 600)


In [28]:
np.nanmax(sst),np.nanmin(sst)

(31.901611, -1.8474731)

## 3DS

In [29]:
data_3ds = nc.Dataset(r'D:/data/3DTS/3DS_2005_2015_month.nc')
sali_depth = data_3ds['so'][:].data
print(sali_depth.shape)

(132, 43, 280, 600)


In [30]:
depths = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
         450,500,550,600,700,800,900,1000,1100,1200,1300,1400,1500,1750,2000]
depths_use = [5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
              500,600,700,800,900,1000,1100,1300,1500,1750,2000]

In [ ]:
sali_depth_use = []
selected_layers = []
for i, depth in enumerate(depths):
    if depth in depths_use:
        selected_layers.append(sali_depth[:, i:i+1, :, :])
if selected_layers:
    sali_depth_use = np.concatenate(selected_layers, axis=1)
else:
    sali_depth_use = None

In [33]:
print(sali_depth_use.shape)
sali_depth_use = np.transpose(sali_depth_use,(0,2,3,1))
print(sali_depth_use.shape)

(132, 33, 280, 600)
(132, 280, 600, 33)


## 数据合并

In [34]:
print(ssh.shape,sst.shape,uwnd.shape,vwnd.shape,sss.shape)
cat_data =np.stack((ssh,sst,uwnd,vwnd,sss),axis=3)
cat_data.shape

(132, 280, 600) (132, 280, 600) (132, 280, 600) (132, 280, 600) (132, 280, 600)


(132, 280, 600, 5)

In [35]:
cat_data_ss_depth = np.concatenate((cat_data,sali_depth_use),axis=-1)
print(cat_data_ss_depth.shape)

(132, 280, 600, 38)


# 验证集和测试集的划分

In [36]:
test_num = 120
train = cat_data_ss_depth[:test_num]
test = cat_data_ss_depth[test_num:]
print('train:',train.shape,'test:',test.shape)

train: (120, 280, 600, 38) test: (12, 280, 600, 38)


## 去掉合并数据中的nan


In [37]:
train_flatten = train.reshape(-1,38)
train_flatten[train_flatten == 32767] = np.nan
print('train删除nan值之前',train_flatten.shape)
nan_mask = np.isnan(train_flatten)
train_without_nan = train_flatten[~nan_mask.any(axis=1)]  #只要有一个是True就会返回True, 此处True代表nan，故求反
print('train删除nan值之后',train_without_nan.shape)

train删除nan值之前 (20160000, 38)
train删除nan值之后 (12534840, 38)


In [38]:
test_flatten =  test.reshape(-1,38)
test_flatten[test_flatten == 32767] = np.nan
print('test删除nan值之前',test_flatten.shape)
nan_mask = np.isnan(test_flatten)
test_without_nan = test_flatten[~nan_mask.any(axis=1)]
print('test删除nan值之后',test_without_nan.shape)

test删除nan值之前 (2016000, 38)
test删除nan值之后 (1253616, 38)


## 划分训练数据和目标数据

In [39]:
x_test = test_without_nan[:,:5]
y_test = test_without_nan[:,5:]

x_train_val = train_without_nan[:,:5]
y_train_val = train_without_nan[:,5:]

In [40]:
print('x_test.shape:',x_test.shape)
print('y_test.shape',y_test.shape)
print('x_train_val',x_train_val.shape)
print('y_train_val',y_train_val.shape)

x_test.shape: (1253616, 5)
y_test.shape (1253616, 33)
x_train_val (12534840, 5)
y_train_val (12534840, 33)


In [41]:
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.25, random_state=100)
x_train.shape,x_test.shape,x_val.shape,y_train.shape,y_test.shape,y_val.shape

# 特征归一化

In [44]:
scaler_f = StandardScaler()
x_train = scaler_f.fit_transform(x_train) 
x_test = scaler_f.transform(x_test)
x_val = scaler_f.transform(x_val)

scaler_l = StandardScaler()
y_train = scaler_l.fit_transform(y_train)
y_test = scaler_l.transform(y_test)
y_val = scaler_l.transform(y_val)

x_train.shape,x_test.shape,x_val.shape,y_train.shape,y_test.shape,y_val.shape

# 定义网络结构

In [46]:
def create_model():   
    model = models.Sequential([
        layers.Dense(200, activation='tanh', input_shape=(5,)),
        layers.Dense(15, activation='tanh'),
        layers.Dense(8, activation='tanh'),
        layers.Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model 


## 编译模型

In [ ]:
model = create_model()
model.summary()

## 模型训练

### 对所有层进行训练

In [49]:
def BPNN_model(x_train,y_train,x_val,y_val,name):
    model = create_model()
    model.compile(optimizer='adam', loss='mean_squared_error')
    checkpoint_path=name
    keras_callbacks   = [
          EarlyStopping(monitor='val_loss', patience=30, mode='min', min_delta=0.001),
          ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
    ]
    model.fit(x_train, y_train, validation_data=(x_val,y_val),
        epochs=200, batch_size=5120, verbose=2, callbacks=keras_callbacks)

In [ ]:
depth=[5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,500,600,700,800,900,1000,1100,1300,1500,1750,2000]
for i in range(0,33):
    BPNN_model(x_train, y_train[:,i:i+1],x_val, y_val[:,i:i+1],
               'D:/BPNN/sail/BPNN_model_'+str(depth[i])+'m.h5')

In [ ]:
history = model.fit(x_train, y_train[:,0:1], validation_data=(x_val,y_val[:,0:1]),
                    epochs=200, batch_size=5210, verbose=2, callbacks=keras_callbacks)

## 模型评估

In [52]:
import math
def acc(actual, predicted):
    pred_avg = np.average(predicted)

    act_avg = np.average(actual)
    diff_pred = predicted - pred_avg
    diff_act = actual - act_avg
    numerator = np.mean(np.sum(diff_pred*diff_act, axis=0))
    denominator = math.sqrt(np.mean(np.sum(diff_pred**2, axis=0)) * np.mean(np.sum(diff_act**2, axis=0)))
    ret_val = numerator/denominator
    return (100 * ret_val)

def look_loss(history):
    plt.figure(figsize=(6.3, 2.5), dpi = 100)
    plt.plot(history.history['loss'], label='training data')
    plt.plot(history.history['val_loss'], label='validation data')
    plt.title('Loss')
    plt.ylabel('MSE Loss')
    plt.xlabel('Number of epochs')
    plt.grid()
    plt.legend(loc="upper right")
    plt.show()

def all_estimate(best_model,x_test,y_test):
    testPred_1 = best_model.predict(x_test)
    a = y_test
    y_test_p = a.reshape(-1,1)
    testPred_p = testPred_1.reshape(-1,1)

    rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
    print('Test RMSE: %.3f' % rmse)

    r2 = r2_score(y_test_p,testPred_p)
    print("R² score: %.4f" %r2)  
    #print("%.4f" % r2)
    

def respective_estimate(best_model,x_test,y_test):
    re2_list = []
    rmse_list = []
    for i in range(6):
        testPred = best_model.predict(x_test[i:i+1])
        a = y_test[i:i+1]
        y_test_p = a.reshape(-1,1)
        testPred_p = testPred.reshape(-1,1)
        r2 = r2_score(y_test_p,testPred_p)
        rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
        re2_list.append(r2)
        rmse_list.append(rmse)    
    return re2_list,rmse_list

### 结果评估

In [ ]:
look_loss(history)

In [ ]:
depth=[5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,500,600,700,800,900,1000,1100,1300,1500,1750,2000]
result = [] 
for i in range(0,33):
    best_model = tf.keras.models.load_model('D:/BPNN/sail/BPNN_model_'+str(depth[i])+'m.h5')
    print('预测第'+str(depth[i])+'m盐度的评价')
    all_estimate(best_model,x_test,y_test[:,i:i+1])
    testPred = best_model.predict(x_test)
    result.append(testPred)

### 反归一化结果评价

In [ ]:
result = np.array(result)
print(result.shape)
result = np.transpose(result,(1,2,0))
print(result.shape)
result = result.reshape(-1,33)
print(result.shape)

In [ ]:
result_unscaled = scaler_l.inverse_transform(result)
y_test_unscaled = scaler_l.inverse_transform(y_test)
result_unscaled.shape,y_test.shape

In [ ]:
for i in range(0,15):
    print('预测第'+str(depth[i])+'m盐度的评价')
    rmse = sqrt(mean_squared_error(y_test_unscaled[:,i],result_unscaled[:,i]))
    print('Test RMSE: %.3f' % rmse)
    r2 = r2_score(y_test_unscaled[:,i],result_unscaled[:,i])
    print('R² score:',r2)

# 预测结果可视化

## 为预测值填充nan

In [73]:
result.shape,test_flatten.shape 

(2016000, 38)

In [74]:
nan_mask = np.isnan(test_flatten)
nan_masks = nan_mask.any(axis=1)    
print(nan_masks.shape)

(2016000,)


In [77]:
result_with_nan = np.full(((nan_masks.shape[0]),33), np.nan)  
result_with_nan.shape,result.shape 
np.count_nonzero(nan_masks)

In [82]:
num = 0
for i,nan_mask in enumerate(nan_masks):   
    if (nan_mask == False): 
        result_with_nan[i] = result_unscaled[num]
        num = num + 1       
print(num)

1256760


## 为真实值填充nan 

In [83]:
y_test_with_nan = np.full(((nan_masks.shape[0]),33), np.nan) 

In [85]:
num = 0 
for i,nan_mask in enumerate(nan_masks):
    if (nan_mask == False):
        y_test_with_nan[i] = y_test[num]
        num = num + 1     
print(num)

1256760


In [ ]:
result_with_nan_reshape = result_with_nan.reshape((12,280,600,33))
y_test_with_nan_reshape = y_test_with_nan.reshape((12,280,600,33))
print(result_with_nan_reshape.shape , y_test_with_nan_reshape.shape)

In [ ]:
result_with_nan_reshape = np.transpose(result_with_nan_reshape,(0,3,1,2))  
y_test_with_nan_reshape = np.transpose(y_test_with_nan_reshape,(0,3,1,2))
print(result_with_nan_reshape.shape,y_test_with_nan_reshape.shape)

# 结果可视化

In [90]:
lat = []
lon = []
time = []
lon.append(data_ssh['lon'][:].data)
lat.append(data_ssh['lat'][:].data)
start_year = 2015
end_year =2015
time = []
date_range = pd.date_range(datetime(start_year,1,1),datetime(end_year+1,1,1),freq='1M')
len(date_range)
for i in range(len(date_range)):
    time.append(i)

In [91]:
new_NC = nc.Dataset("D:/BPNN/result_sali_BPNN_5-2000m.nc", 'w', format='NETCDF4')
'''
定义维度，后一个参数表示维度的长度，因为是合并的同一个产品的数据，所以是统一
的,注意维度的长度一定要和读入的数据匹配
'''
new_NC.createDimension('lat', len(lat[0]))
new_NC.createDimension('lon', len(lon[0]))
new_NC.createDimension('depth', 33)
new_NC.createDimension('time', len(time))

new_NC.createVariable('lat', 'f', ("lat"))
new_NC.createVariable('lon', 'f', ("lon"))
new_NC.createVariable('depth', 'f', ("depth"))
new_NC.createVariable('true_sss', 'f',("time","depth","lat","lon"))
new_NC.createVariable('pred_sss', 'f',("time","depth","lat","lon"))

time_var = new_NC.createVariable('time', 'f4',("time"))
time_var.units = 'months since 2015-01-15'
time_var.long_name = 'Months in Monthly Means'
time_var.axis = 'T'

new_NC.variables['lat'][:] = lat[0]
new_NC.variables['lon'][:] = lon[0]
new_NC.variables['time'][:] = np.array(time)
new_NC.variables['depth'][:] = depth

new_NC.variables['true_sss'][:]=np.array(y_test_with_nan_reshape)
new_NC.variables['pred_sss'][:]=np.array(result_with_nan_reshape)

new_NC.close()